<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection_Datasets_For_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection_Datasets_For_YOLO

## Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Explore Datasets

### Load Datasets

In [3]:
!unzip /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/zips/liver-ultrasound-detection.zip

Streaming output truncated to the last 5000 lines.
  inflating: val/val/annotations/95431.txt  
  inflating: val/val/annotations/95442.txt  
  inflating: val/val/annotations/95505.txt  
  inflating: val/val/annotations/95579.txt  
  inflating: val/val/annotations/956.txt  
  inflating: val/val/annotations/95620.txt  
  inflating: val/val/annotations/95632.txt  
  inflating: val/val/annotations/95685.txt  
  inflating: val/val/annotations/95697.txt  
  inflating: val/val/annotations/9571.txt  
  inflating: val/val/annotations/95729.txt  
  inflating: val/val/annotations/9573.txt  
  inflating: val/val/annotations/9576.txt  
  inflating: val/val/annotations/95785.txt  
  inflating: val/val/annotations/95827.txt  
  inflating: val/val/annotations/95851.txt  
  inflating: val/val/annotations/95897.txt  
  inflating: val/val/annotations/96013.txt  
  inflating: val/val/annotations/96048.txt  
  inflating: val/val/annotations/96208.txt  
  inflating: val/val/annotations/96316.txt  
  inflati

### Explore Train Images Directory

In [ ]:
import os

In [ ]:
n_images = len(os.listdir('/content/train/train/images'))
n_lesions = len(os.listdir('/content/train/train/annotations'))
n_non_lesions = n_images - n_lesions

print('num of datas =' , n_images , 'num of lesion =', n_lesions , 'num of non lesion' , n_non_lesions)

num of datas = 14448 num of lesion = 7222 num of non lesion 7226


### Explore Train Annotations Directory

In [ ]:
annotations_path = '/content/train/train/annotations'

with open (f'{annotations_path}/{100022}.txt' , 'r') as file :

    file_content = file.read().split(' ')

    print(f'class = {file_content[0]}\ncenter_x = {file_content[1]}\ncenter_y = {file_content[2]}\nbox_width / image_width = {file_content[3]}\nbox_height / image_height = {file_content[4]}')

class = 2
center_x = 0.4601851851851852
center_y = 0.30185185185185187
box_width / image_width = 0.08703703703703704
box_height / image_height = 0.12222222222222222



## Setup Format Datasets For YOLOv10

### Define Path Of Directories

In [ ]:
import os
import pandas as pd

In [ ]:
train_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/images'
validation_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/validation/images'
test_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/test/images'

In [ ]:
train_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/train/labels'
validation_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/datasets/validation/labels'

### Managing With Annotation Mapping File

In [ ]:
annotation_mapping_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_mapping.csv'
annotation_mapping = pd.read_csv(annotation_mapping_path)

In [ ]:
annotation_mapping

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
19341,1160.jpg,NaN,mobile,mobile_negative
19342,153.jpg,NaN,mobile,mobile_negative
19343,130705.jpg,NaN,mobile,mobile_negative
19344,812.jpg,NaN,mobile,mobile_negative


In [ ]:
annotation_mapping_trainable = annotation_mapping[annotation_mapping['Annotation File'].isnull() == False]

In [ ]:
annotation_mapping_trainable

,Image File,Annotation File,Source,Type
0,24084.jpg,24084.txt,machine,machine_positive
1,97528.jpg,97528.txt,machine,machine_positive
2,54563.jpg,54563.txt,machine,machine_positive
3,115211.jpg,115211.txt,machine,machine_positive
4,71913.jpg,71913.txt,machine,machine_positive
...,...,...,...,...
18662,31309.jpg,31309.txt,mobile,mobile_positive
18663,103330.jpg,103330.txt,mobile,mobile_positive
18664,141103.jpg,141103.txt,mobile,mobile_positive
18665,9070.jpg,9070.txt,mobile,mobile_positive


In [ ]:
print(f'num of all files = {len(annotation_mapping)} num of trainable files = {len(annotation_mapping_trainable)}')

num of all files = 19346 num of trainable files = 9667


### Create annotation_for_datasets.csv file

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
annotation_for_train_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [ ]:
# loop in train
train_dataset_path = '/content/train/train'

train_dataset_annotations = os.listdir(f'{train_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{train_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_train_datasets['file_name'].append(name)

    class_ = 1 if f'{name}.txt' in train_dataset_annotations else 0
    annotation_for_train_datasets['class'].append(class_)

    image = Image.open(f'{train_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_train_datasets['width'].append(shape[0])
    annotation_for_train_datasets['height'].append(shape[1])

14448it [02:45, 87.04it/s]


In [ ]:
annotation_for_train_datasets_df = pd.DataFrame.from_dict(annotation_for_train_datasets)
annotation_for_train_datasets_df.set_index('file_name', inplace=True)

In [ ]:
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [ ]:
annotation_for_train_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv')

In [ ]:
annotation_for_train_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_train_datasets.csv' , index_col = 'file_name')
annotation_for_train_datasets_df

,width,height,class
file_name,,,
98356,782,1042,1
74796,810,1080,0
121413,810,1080,1
148426,540,720,1
122997,540,720,0
...,...,...,...
69995,540,720,1
116763,810,1080,0
106056,810,1080,1


In [ ]:
annotation_for_validation_datasets = {

    'file_name' : [] ,
    'width' : [] ,
    'height' : [] ,
    'class' : [] ,

}
#class 0 -> no lesion 1 -> lesion

In [ ]:
# loop in validation
validation_dataset_path = '/content/val/val'

validation_dataset_annotations = os.listdir(f'{validation_dataset_path}/annotations')

for idx , file_name in tqdm(enumerate(os.listdir(f'{validation_dataset_path}/images'))) :

    name = file_name.split('.')[0]

    annotation_for_validation_datasets['file_name'].append(name)
    class_ = 1 if f'{name}.txt' in validation_dataset_annotations else 0

    annotation_for_validation_datasets['class'].append(class_)

    image = Image.open(f'{validation_dataset_path}/images/{file_name}')

    shape = np.array(image).shape

    annotation_for_validation_datasets['width'].append(shape[0])
    annotation_for_validation_datasets['height'].append(shape[1])

4898it [02:44, 29.82it/s]


In [ ]:
annotation_for_validation_datasets_df = pd.DataFrame.from_dict(annotation_for_validation_datasets)
annotation_for_validation_datasets_df.set_index('file_name', inplace=True)

In [ ]:
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


In [ ]:
annotation_for_validation_datasets_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv')

In [ ]:
annotation_for_validation_datasets_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/annotations/annotation_for_validation_datasets.csv' , index_col = 'file_name')
annotation_for_validation_datasets_df

,width,height,class
file_name,,,
27908,540,720,0
77411,540,720,0
48899,540,720,1
23485,810,1080,1
129834,614,816,0
...,...,...,...
86856,3024,4032,1
1684,3024,4032,0
10519,660,880,0


### Create Images / Labels Folder

In [4]:
import cv2
import os
from tqdm import tqdm

In [8]:
train_dataset_images_path = '/content/train/train/images'
validation_dataset_images_path = '/content/val/val/images'
test_dataset_images_path = '/content/test/test/images'

train_dataset_labels_path = '/content/train/train/annotations'
validation_dataset_labels_path = '/content/val/val/annotations'

In [9]:
new_train_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/train/images'
new_validation_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/validation/images'
new_test_dataset_images_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/test/images'

new_train_dataset_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/train/labels'
new_validation_dataset_labels_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/liver_ultrasound_datasets/yolo_format_datasets/validation/labels'

In [ ]:
for file_name in tqdm(os.listdir(train_dataset_images_path)) :

    image = cv2.imread(f'{train_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_train_dataset_images_path}/{file_name}' , image)

100%|██████████| 14448/14448 [11:17<00:00, 21.32it/s]


In [ ]:
for file_name in tqdm(os.listdir(validation_dataset_images_path)) :

    image = cv2.imread(f'{validation_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_validation_dataset_images_path}/{file_name}' , image)

100%|██████████| 4898/4898 [12:36<00:00,  6.47it/s]


In [ ]:
for file_name in tqdm(os.listdir(test_dataset_images_path)) :

    image = cv2.imread(f'{test_dataset_images_path}/{file_name}')
    cv2.imwrite(f'{new_test_dataset_images_path}/{file_name}' , image)

100%|██████████| 5153/5153 [33:36<00:00,  2.55it/s]


In [10]:
label_files = os.listdir(train_dataset_labels_path)

for file_name in tqdm(os.listdir(train_dataset_images_path)) :

    file_name = file_name.split('.')[0]

    if (file_name + '.txt' in label_files) :

        with open(f'{train_dataset_labels_path}/{file_name}.txt' , 'r') as src_file :

            with open(f'{new_train_dataset_labels_path}/{file_name}.txt' , 'w') as dest_file :

                dest_file.write(src_file.read())

    else :

        with open(f'{new_train_dataset_labels_path}/{file_name}.txt' , 'w') as dest_file :

                dest_file.write('')

100%|██████████| 14448/14448 [01:01<00:00, 235.68it/s]


In [11]:
label_files = os.listdir(validation_dataset_labels_path)

for file_name in tqdm(os.listdir(validation_dataset_images_path)) :

    file_name = file_name.split('.')[0]

    if (file_name + '.txt' in label_files) :

        with open(f'{validation_dataset_labels_path}/{file_name}.txt' , 'r') as src_file :

            with open(f'{new_validation_dataset_labels_path}/{file_name}.txt' , 'w') as dest_file :

                dest_file.write(src_file.read())

    else :

        with open(f'{new_validation_dataset_labels_path}/{file_name}.txt' , 'w') as dest_file :

                dest_file.write('')

100%|██████████| 4898/4898 [00:21<00:00, 229.23it/s]


In [12]:
print(len(os.listdir(new_train_dataset_labels_path)) , len(os.listdir(new_validation_dataset_labels_path)))

14448 4898
